In [2]:
import requests
import json
#from typing import Tuple
import time
import random
from login import Login
import pandas as pd
import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/ 537.36 Edg/107.0.1418.62'
}


In [5]:
with open('following.json', 'r') as f:
    following_list = json.loads(f.read())
len(following_list)

830

In [7]:
login = Login()
url, token = login.apply_QR_code()
login.generate_QR_code(url)
input('已扫码')
bilibili_session = requests.Session()
code = login.do_web_login(bilibili_session, token)
if code == 0:
    print('登录成功')
elif code == 86038:
    print('二维码失效')
elif code == 86090 or code == 86101:
    print('请扫码并确认')
else:
    print(f'错误，code:{code}')


█████████████████████████████████████████████████████
█████████████████████████████████████████████████████
████ ▄▄▄▄▄ ██▀█▀▄ ▄▄▀▄▀ █▄█▀▀▄▀ █ ▄  ██▀▄█ ▄▄▄▄▄ ████
████ █   █ █  ▀▀▄ ▄▀█▄ █▀▀▀▀▄▄▀▄▄▄ ▀▄▄█ ▀█ █   █ ████
████ █▄▄▄█ █ ▄██▀ ▄▄▄ █▄ ▄▄▄  ▀▀▀█▄▄█▀▄▄▄█ █▄▄▄█ ████
████▄▄▄▄▄▄▄█ █▄█▄▀▄▀ █▄▀ █▄█ ▀ ▀ ▀ █ ▀▄█ █▄▄▄▄▄▄▄████
████ █▄▄▄ ▄▀▀▄       ▄▀   ▄▄ ▀███   ▀▄█▀▀█▄▄   ▀▀████
████▄▄▄█ █▄ ▀ █ ▀█▄▄█ ▀▄█ █▀ ▀   ▄▀▀  ▄▄▀▄▄▄▀▄ ▄█████
████▄█▀▄▀▄▄▄▄▄█▄█▄█▀ █▄▄██  ██▄▀▀ ▀▄▀█▄ ▀██ ▀▀▀▀ ████
████▄▀ ▀ ▄▄ █▀█ ▄█▄█ ▄▄▄▄▄ ▀ █▄█▀█▀█▀█ ▄ ▀▄▀▀▀ █▀████
████▄ █▀ ▄▄ █▀ ▄ ▄▄█▄▄▄▀▄ ▀█▄▀▄▄█▀▀▄██▄ ▀▀▄▄██▀█▀████
████▄ ▄  █▄ ▄ ▀ █▀█▄█▄▄▀▄▀█ ▄█▄▀▀ ██▀▄▄ █ ▄▄▀   █████
████▄ ▀▄ ▄▄▄ ▀ █▀▀▀▀ ▀▄▀ ▄▄▄ ▀▄█▀ ▀▄█ █  ▄▄▄ ▀▀▀ ████
████ ▀   █▄█ ▄▀ ▀▀█▄▀▀   █▄█ █ ▀▀██▄▀▀▄█ █▄█ ▀ ██████
████ ▀ ▀ ▄▄▄▄▀▀  █▄▀█▀ ▀   ▄  █▀▀▄  ▀██▀ ▄▄▄▄ ██ ████
████▄██▄▄▀▄ ▄██ █ ▀█▄  ▀▄▀ ▀▀▀  ▀ ▀  ▀▄▀▄▀▀ █  ▄█████
████▀  ▀▀▀▄   ▄▄▄▀ █▀█ █▀▄▀▄▄ ▄▄▀   ▀█▄▀ █ █▄▀▀▄▀████
████  █▄ █▄ ▄▀█▄▄▀▀██ ▀ ▀▄▀▄▀▄▄ ▀▀▀  █▄▀▄█▀ █▄ ▄█████
████▀ ▀  █▄▀▀▀█ █ ▀ ▀▄█ ▀▄▀ 

In [4]:
# Login before

def get_up_stat(session: requests.Session, mid: int) -> dict:
    info = {}
    res = session.get(url='https://api.bilibili.com/x/space/upstat',
                      headers=headers, params={'mid': mid})
    res = json.loads(res.text)
    data = res['data']
    view = data['archive']['view']
    likes = data['likes']
    info['view'] = view
    info['likes'] = likes
    return info


In [47]:
get_up_stat(bilibili_session, 258150656)


{'view': 185145410, 'likes': 8263343}

In [103]:
def get_up_videos(mid: int) -> dict:
    while True:
        res = requests.get(url='http://api.bilibili.com/x/space/arc/search',
                           headers=headers, params={'mid': mid, 'pn': 1, 'ps': 1})
        res = json.loads(res.text)
        data = res['data']
        code = res['code']
        if code == 0:
            break
        time.sleep(random.randint(1, 3))
    total = data['page']['count']

    if total != 0:
        fields = data['list']['tlist']
        fields_count = {value['name']: value['count']
                        for value in fields.values()}
        newest = data['list']['vlist'][0]['created']
        newest = time.strftime('%Y-%m-%d', time.localtime(newest))
        page = total // 51 + 1
        while True:
            res = requests.get(url='http://api.bilibili.com/x/space/arc/search',
                               headers=headers, params={'mid': mid, 'pn': page, 'ps': 50})
            res = json.loads(res.text)
            code = res['code']
            if code == 0:
                break
            time.sleep(random.randint(1, 3))
        data = res['data']
        oldest = data['list']['vlist'][-1]['created']
        oldest = time.strftime('%Y-%m-%d', time.localtime(oldest))
    else:
        fields_count = {'无': 0}
        newest = '1970-1-1'
        oldest = '1970-1-1'

    info = {
        'total': total,
        'fields_count': fields_count,
        'newest': newest,
        'oldest': oldest
    }
    return info


get_up_videos(258150656)
get_up_videos(672328094)


{'total': 258,
 'fields_count': {'动画': 214, '舞蹈': 6, '资讯': 1, '音乐': 30, '游戏': 7},
 'newest': '2022-12-12',
 'oldest': '2020-12-14'}

In [30]:
df = pd.read_csv('./following-results.csv')
df = df.iloc[:, 1:]
df.head()


,名称,UID,关注时间,特别关注,头像链接,粉丝数,主页
0,-小拉-,41759,2022-12-08,否,https://i2.hdslb.com/bfs/face/ce83957d60ce9800...,286680,https://space.bilibili.com/41759
1,Biloneness,399763875,2022-12-07,否,http://i0.hdslb.com/bfs/face/70e5621d3ca80fe1e...,12,https://space.bilibili.com/399763875
2,无极恋画,102541434,2022-12-06,否,https://i0.hdslb.com/bfs/face/8297704cd928e35a...,86914,https://space.bilibili.com/102541434
3,修理铺韩师傅,381801980,2022-12-06,否,https://i2.hdslb.com/bfs/face/6dcffadade0fd367...,13897,https://space.bilibili.com/381801980
4,VitoShen涤非,3493075427002798,2022-12-06,是,https://i1.hdslb.com/bfs/face/e8c393b710c5bd1c...,639,https://space.bilibili.com/3493075427002798


In [106]:
view = []
likes = []
fields = []
newest = []
oldest = []
total = []
for mid in tqdm.tqdm(df['UID']):
    upstat = get_up_stat(bilibili_session, mid)
    view.append(upstat['view'])
    likes.append(upstat['likes'])
    time.sleep(random.randint(1, 2))
    video_info = get_up_videos(mid)
    newest.append(video_info['newest'])
    oldest.append(video_info['oldest'])
    total.append(video_info['total'])
    fields_count = video_info['fields_count']
    # use the first two fields with the largest video number, regardless of the same
    if len(fields_count) > 1:
        main_fields = sorted(fields_count.items(),
                             key=lambda x: x[1], reverse=True)[0:2]
        first_field = main_fields[0][0]
        if main_fields[1][1] / video_info['total'] >= 0.33:
            second_field = main_fields[1][0]
        else:
            second_field = ''
    else:
        first_field = sorted(fields_count.items(),
                             key=lambda x: x[1], reverse=True)[0][0]
        second_field = ''
    if second_field != '':
        fields.append(first_field+','+second_field)
    else:
        fields.append(first_field)
    time.sleep(random.randint(1, 2))


100%|██████████| 830/830 [48:04<00:00,  3.48s/it]


In [107]:
len(fields), fields[0:10]


(830, ['生活', '生活', '知识', '生活', '科技', '舞蹈,动画', '知识', '生活', '游戏', '游戏'])

In [108]:
len(total) == len(fields) == len(newest) == len(
    oldest) == len(view) == len(likes)


True

In [119]:
from datetime import datetime
update = []
for up in newest:
    date = datetime.strptime(up, '%Y-%m-%d')
    days = (datetime.now() - date).days
    if days > 365:
        update.append('停更')
    elif days > 30 * 6:
        update.append('停更风险')
    elif days > 30 * 3:
        update.append('暂时停更')
    elif days > 30:
        update.append('更新缓慢')
    else:
        update.append('仍在更新')
update[0:10]


['仍在更新',
 '停更风险',
 '仍在更新',
 '仍在更新',
 '仍在更新',
 '更新缓慢',
 '仍在更新',
 '仍在更新',
 '仍在更新',
 '仍在更新']

In [120]:
addition = pd.DataFrame({'视频总数': total,
                         '总播放量': view,
                         '总获赞数': likes,
                         '分区': fields,
                         '最早视频': oldest,
                         '最新视频': newest,
                         '更新状态': update})
addition.head()


,视频总数,总播放量,总获赞数,分区,最早视频,最新视频,更新状态
0,30,12865058,723059,生活,2022-06-21,2022-12-04,仍在更新
1,2,10,10,生活,2022-06-07,2022-06-07,停更风险
2,94,2470178,137713,知识,2018-02-27,2022-12-09,仍在更新
3,4,724358,35531,生活,2022-10-23,2022-12-04,仍在更新
4,14,16251,746,科技,2022-11-29,2022-12-13,仍在更新


In [122]:
df_new = pd.concat([df, addition], axis=1)
df_new.head()


,名称,UID,关注时间,特别关注,头像链接,粉丝数,主页,视频总数,总播放量,总获赞数,分区,最早视频,最新视频,更新状态
0,-小拉-,41759,2022-12-08,否,https://i2.hdslb.com/bfs/face/ce83957d60ce9800...,286680,https://space.bilibili.com/41759,30,12865058,723059,生活,2022-06-21,2022-12-04,仍在更新
1,Biloneness,399763875,2022-12-07,否,http://i0.hdslb.com/bfs/face/70e5621d3ca80fe1e...,12,https://space.bilibili.com/399763875,2,10,10,生活,2022-06-07,2022-06-07,停更风险
2,无极恋画,102541434,2022-12-06,否,https://i0.hdslb.com/bfs/face/8297704cd928e35a...,86914,https://space.bilibili.com/102541434,94,2470178,137713,知识,2018-02-27,2022-12-09,仍在更新
3,修理铺韩师傅,381801980,2022-12-06,否,https://i2.hdslb.com/bfs/face/6dcffadade0fd367...,13897,https://space.bilibili.com/381801980,4,724358,35531,生活,2022-10-23,2022-12-04,仍在更新
4,VitoShen涤非,3493075427002798,2022-12-06,是,https://i1.hdslb.com/bfs/face/e8c393b710c5bd1c...,639,https://space.bilibili.com/3493075427002798,14,16251,746,科技,2022-11-29,2022-12-13,仍在更新


In [123]:
df_new['UID'] = df_new['UID'].astype(str)
df_new.to_excel('following-whole-info.xlsx')
df_new.to_csv('following-whole-info.csv')
